#### Importing Libraries & Loading Data

This notebook is likely to crash the VM. Run the notebook in chunks and save the vectors to a file

In [ ]:
#This ipynb was developed on Google Colab and uses file from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import spacy
import gensim.downloader as api
from gensim.models.keyedvectors import KeyedVectors
from scipy import spatial

In [ ]:
parent_folder = "/content/drive/MyDrive/AAIC/Projects/Quora Question Pairs"
filename = "train_fe_2.csv"
df = pd.read_csv(parent_folder + "/" + filename)

In [ ]:
df.head(2)

,Unnamed: 0,Unnamed: 0.1,id,qid1,qid2,question1,question2,is_duplicate,length_difference,unique_length_difference,...,cwc_min,cwc_max,cnc_min,cnc_max,last_word_equal,first_word_equal,fuzz_ratio,fuzz_partial_ratio,fuzz_token_sort_ratio,fuzz_token_set_ratio
0,0,0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,2,1,...,0.916659,0.785709,0.999980,0.833319,0,1,93,100,93,100
1,1,1,1,3,4,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0,5,4,...,0.699993,0.466664,0.799984,0.399996,0,1,65,73,63,86


In [ ]:
# converting questions to string as a safety measure
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

#### Loading embedding models

In [ ]:
!python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 827.9 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
# nlp = spacy.load('en')
spacy_model = spacy.load('en_core_web_lg')

In [ ]:
# path = api.load("word2vec-google-news-300", return_path=True)
# print(path)

# gensim_model = KeyedVectors.load_word2vec_format(path, binary=True)

#### Checking results of spacy and gensim using cosine similarity

In [ ]:
df[['question1', 'question2']][:10]

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math is div...
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water
5,astrology i am a capricorn sun cap moon and ca...,i am a triple capricorn sun moon and ascendant...
6,should i buy tiago,what keeps childern active and far from phone ...
7,how can i be a good geologist,what should i do to be a great geologist
8,when do you use instead of,when do you use instead of and
9,motorola company can i hack my charter motorol...,how do i hack motorola dcx3400 for free internet


##### Spacy

In [ ]:
# vecs1 = []
# for qu1 in tqdm(list(df['question1'][:10])):
#     doc1 = spacy_model(qu1)
#     # print(doc1)
#     mean_vec1 = np.zeros(len(doc1[0].vector))
#     # print(mean_vec1.shape)
#     for word1 in doc1:
#         # word1vec
#         vec1 = word1.vector
#         # print(f"For {word1} : {vec1}")
#         # print()
#         # print()
#         # compute final vec
#         mean_vec1 += vec1
    
#     # print(mean_vec1)
#     # print()
#     # print()
#     mean_vec1 = mean_vec1/len(qu1.split())
#     # print(mean_vec1)
#     vecs1.append(mean_vec1)
# # print(vecs1)

# from tqdm import tqdm
# vecs2 = []
# for qu2 in tqdm(list(df['question2'][:10])):
#     doc2 = spacy_model(qu2)
#     mean_vec2 = np.zeros(len(doc2[0].vector))

#     for word2 in doc2:
#         vec2 = word2.vector
#         # compute final vec
#         mean_vec2 += vec2

#     mean_vec2 = mean_vec2/len(qu2.split())
#     vecs2.append(mean_vec2)
# # print(vecs2)

In [ ]:
# result = [1 - spatial.distance.cosine(vecs1[q_index], vecs2[q_index]) for q_index in range(len(vecs1))]
# print(f"Cosine similarity for first 10 questions using spacy : {result}")

##### Gensim

In [ ]:
# # average Word2Vec
# w2v_words = list(gensim_model.wv.vocab)
# que_vectors1 = [];
# for que in tqdm(list(df['question1'][:10])):
#     que_vec = np.zeros(300)
#     cnt_words =0;
#     for word in que:
#         if word in w2v_words:
#             vec = gensim_model.wv[word]
#             que_vec += vec
#             cnt_words += 1
#     if cnt_words != 0:
#         que_vec /= cnt_words
#     que_vectors1.append(que_vec)

# # average Word2Vec
# w2v_words = list(gensim_model.wv.vocab)
# que_vectors2 = [];
# for que in tqdm(list(df['question2'][:10])):
#     que_vec = np.zeros(300) 
#     cnt_words =0;
#     for word in que:
#         if word in w2v_words:
#             vec = gensim_model.wv[word]
#             que_vec += vec
#             cnt_words += 1
#     if cnt_words != 0:
#         que_vec /= cnt_words
#     que_vectors2.append(que_vec)

In [ ]:
# result = [1 - spatial.distance.cosine(que_vectors1[q_index], que_vectors2[q_index]) for q_index in range(len(vecs1))]
# print(f"Cosine distances for first 10 similarity using gensim : {result}")

#### Converting question1 and question2 into vectors using spacy

Following code block can take around 2-3 hours to run. The output vectors are saved for future use.

##### question1

In [ ]:
q1_vectors = []
for question in tqdm(list(df['question1'])):
    doc = spacy_model(question)
    mean_vec = np.zeros(len(doc[0].vector))
    for word in doc:
        # wordvec
        vec = word.vector
        mean_vec += vec

    mean_vec = mean_vec/len(question.split())
    q1_vectors.append(mean_vec)

  1%|          | 3293/404290 [00:24<44:39, 149.68it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 404290/404290 [49:16<00:00, 136.76it/s]


In [ ]:
print("Sanity check")
print(f"Total questions : {len(df['question1'])}")
print(f"Total vectors : {len(q1_vectors)}")
# print("Vector for first question : ")
# q1_vectors[0]

Sanity check
Total questions : 404290
Total vectors : 404290


In [ ]:
q1_vectors = np.array(q1_vectors)
np.savetxt(parent_folder + "/" + "q1_vectors.csv", q1_vectors, fmt='%f', delimiter=",")

In [ ]:
column_names = ['q1_'+str(i) for i in range(1,301)]
df = pd.concat([df, pd.read_csv(parent_folder + "/" + "q1_vectors.csv", names=column_names)], axis='columns')

##### question2

In [ ]:
q2_vectors = []
for question in tqdm(list(df['question2'])):
    doc = spacy_model(question)
    mean_vec = np.zeros(len(doc[0].vector))
    for word in doc:
        # wordvec
        vec = word.vector
        mean_vec += vec

    mean_vec = mean_vec/len(question.split())
    q2_vectors.append(mean_vec)

  5%|▍         | 20066/404290 [02:23<48:46, 131.27it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 404290/404290 [48:57<00:00, 137.62it/s]


In [ ]:
print("Sanity check")
print(f"Total questions : {len(df['question2'])}")
print(f"Total vectors : {len(q2_vectors)}")
# print("Vector for first question : ")
# q2_vectors[0]

Sanity check
Total questions : 404290
Total vectors : 404290


In [ ]:
q2_vectors = np.array(q2_vectors)
np.savetxt(parent_folder + "/" + "q2_vectors.csv", q2_vectors, fmt='%f', delimiter=",")

In [ ]:
column_names = ['q2_'+str(i) for i in range(1,301)]
df = pd.concat([df, pd.read_csv(parent_folder + "/" + "q2_vectors.csv", names=column_names)], axis='columns')

#### Calculating cosine similarity

In [ ]:
total = df.shape[0]
chunk_size = 10000
total_chunks = total//chunk_size
remainder = total%chunk_size

In [ ]:
parent_folder = "/content/drive/MyDrive/AAIC/Projects/Quora Question Pairs"
cosine_similarity = []
for chunk in tqdm(range(total_chunks)):
    skip_header = chunk*chunk_size
    skip_footer = total - ((chunk*chunk_size) + chunk_size)

    #print(skip_header, skip_footer)
    q1 = np.genfromtxt(parent_folder + "/" + "q1_vectors.csv", delimiter=',', skip_header=skip_header, skip_footer=skip_footer)
    q2 = np.genfromtxt(parent_folder + "/" + "q2_vectors.csv", delimiter=',', skip_header=skip_header, skip_footer=skip_footer)
    result = [1 - spatial.distance.cosine(q1[q_index], q2[q_index]) for q_index in range(len(q1))]
    cosine_similarity.extend(result)

skip_header = total_chunks*chunk_size
q1 = np.genfromtxt(parent_folder + "/" + "q1_vectors.csv", delimiter=',', skip_header=skip_header)
q2 = np.genfromtxt(parent_folder + "/" + "q2_vectors.csv", delimiter=',', skip_header=skip_header)
result = [1 - spatial.distance.cosine(q1[q_index], q2[q_index]) for q_index in range(len(q1))]
cosine_similarity.extend(result)

 18%|█▊        | 7/40 [05:03<23:08, 42.06s/it]/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 40/40 [18:03<00:00, 27.09s/it]


In [ ]:
print("Sanity check")
vecs1 = []
for qu1 in tqdm(list(df['question1'][100:110])):
    doc1 = spacy_model(qu1)
    mean_vec1 = np.zeros(len(doc1[0].vector))
    for word1 in doc1:
        vec1 = word1.vector
        mean_vec1 += vec1

    mean_vec1 = mean_vec1/len(qu1.split())
    vecs1.append(mean_vec1)

vecs2 = []
for qu2 in tqdm(list(df['question2'][100:110])):
    doc2 = spacy_model(qu2)
    mean_vec2 = np.zeros(len(doc2[0].vector))

    for word2 in doc2:
        vec2 = word2.vector
        mean_vec2 += vec2

    mean_vec2 = mean_vec2/len(qu2.split())
    vecs2.append(mean_vec2)

result = [1 - spatial.distance.cosine(vecs1[q_index], vecs2[q_index]) for q_index in range(len(vecs1))]
print(f"Cosine similarity for random 10 questions using spacy : {result}")

print(f"From the calculated list : {cosine_similarity[100:110]}")

In [ ]:
df['cosine_similarity'] = cosine_similarity

In [ ]:
pd.set_option('display.max_columns', None)
df.head(2)

,Unnamed: 0,Unnamed: 0.1,id,qid1,qid2,question1,question2,is_duplicate,length_difference,unique_length_difference,common_words_count,common_words_share,cwc_min,cwc_max,cnc_min,cnc_max,last_word_equal,first_word_equal,fuzz_ratio,fuzz_partial_ratio,fuzz_token_sort_ratio,fuzz_token_set_ratio,cosine_similarity
0,0,0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,2,1,10,0.434783,0.916659,0.785709,0.999980,0.833319,0,1,93,100,93,100,0.985097
1,1,1,1,3,4,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0,5,4,4,0.200000,0.699993,0.466664,0.799984,0.399996,0,1,65,73,63,86,0.896264


#### Saving dataframe to csv

In [ ]:
df.to_csv(parent_folder + "/" + "train_fe_3.csv")